In [1]:
import pandas as pd

In [2]:
import os
os.getcwd()

'/Users/2010y/opt/fall2022/program-l'

In [3]:
df=pd.read_feather('program_l.feather')

In [4]:
df.head(1)

,subject,from,to,date,body,post_url,followup_url,references_url,related_posts
0,Re: List Move - Completed? - program-l - FreeL...,"""George Bell"" <george@xxxxxxxxxxxxxxxxxxx>","<program-l@xxxxxxxxxxxxxxxx>, <program-l@xxxxx...","Sun, 7 Nov 2004 17:42:11 -0000","Hi all,\n\nFurther to my earlier message today...",https://www.freelists.org/post/program-l/List-...,None,None,None


In [5]:
df['date']

0         Sun, 7 Nov 2004 17:42:11 -0000
1         Mon, 8 Nov 2004 08:35:31 -0500
2         Mon, 8 Nov 2004 15:55:27 -0000
3         Mon, 8 Nov 2004 16:24:19 -0000
4         Mon, 8 Nov 2004 16:25:08 -0000
                      ...               
55485    Fri, 31 Mar 2017 20:18:42 -0400
55486    Fri, 31 Mar 2017 20:48:21 -0400
55487    Wed, 15 Jun 2022 08:03:29 +0200
55488    Sun, 24 Oct 2021 12:58:05 +0530
55489    Tue, 29 Mar 2011 00:16:23 -0500
Name: date, Length: 55490, dtype: object